## Overview

This notebook is intended to:
- Demonstrate how to access time series data for meters and SCADAs using Awesense's Energy Data Model (EDM) SQL API.

Please refer to the [main_concepts.ipynb](../2_main_concepts/main_concepts.ipynb) notebook for a high-level introduction to and simpler examples of the core views and functions available in Awesense's Energy Data Model (EDM).

---

## Set up

In [2]:
import getpass
import plotly.express as px
import plotly.io as pio
import time
import urllib.parse
import pandas as pd

# Allow charts to persist between notebook sessions.
pio.renderers.default='notebook'

**Connection**

Enter the EDM server address and the login credentials provided by Awesense. If you do not have the credentials, or have any trouble connecting, please contact api@awesense.com.
<span style='color:red'> **Please do NOT store the credentials in the notebook, nor share them with anyone.** </span>

In [3]:
edm_address = getpass.getpass(prompt='EDM server address: ')

print('\nEDM login information')
edm_name = getpass.getpass(prompt='Username: ')
edm_password = getpass.getpass(prompt='Password: ')
edm_password = urllib.parse.quote(edm_password)

%load_ext sql
%sql postgresql://$edm_name:$edm_password@$edm_address/edm
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

# Delete the credential variables for security purpose.
del edm_name, edm_password

EDM server address: ········

EDM login information
Username: ········
Password: ········


---

## Time Series
Various temporary views are created below to optimize query performance within the notebook.

### Meter Time Series

*High level approach*
- Create a temporary view `grid_element_metric`, containing information on meters for grid elements and convenient access to the actual data.
- The `grid_get_downstream()` function is used to gather information of meters that are downstream of a specified grid element.
- Time series are retrieved using the `ts_data_source_select()` function.

In [54]:
#Finding branches in NCZ and only in ABC lines

def branchfind(grid_id = 'North Central Zone', grid_element_id = 'L7-97438'):

    Tree = %sql SELECT grid_element_id, phases, terminal1_cn, terminal2_cn FROM grid_get_downstream('{grid_id}', '{grid_element_id}');
    Tree = Tree.DataFrame()

    #Make sure first element has ABC phases
    if sum(Tree[Tree['grid_element_id']==grid_element_id]['phases'] != 'ABC')==1:
        print(f'{grid_element_id} is not ABC')

    #Find the next link
    AA = Tree[Tree['terminal1_cn']==Tree[Tree['grid_element_id']==grid_element_id]['terminal2_cn'].iloc[0]]
    #If there are at least 2 ABC links connected, then break
    if (sum(AA['phases']=='ABC')==2):
        print(f"First branching includes {AA}")

    else:
        # As long as there is only one link
        while (sum(AA['phases']=='ABC')==1):

            #Go on to the next link
            for jj in range(0,len(AA['terminal1_cn'])):
                if AA['phases'].iloc[jj]=='ABC':
                    BB = Tree[(Tree['terminal1_cn']==AA['terminal2_cn'].iloc[jj])]

            AA = BB
    #If there are at least 2 ABC links connected, then break

            if (sum(AA['phases']=='ABC')>1):
                #print(f"First branching includes {AA}")
                return AA



In [184]:
CC = branchfind(grid_element_id='L7-100133')

L7-100133 is not ABC


In [192]:
CC.empty

AttributeError: 'NoneType' object has no attribute 'empty'

In [67]:
def clusfind(grid_id = 'North Central Zone', grid_element_id = 'L7-97438'):

    #Retrieving elements, connections and phases from all elements downstream
    Tree2 = %sql SELECT grid_element_id, terminal1_cn, terminal2_cn, phases FROM grid_get_downstream('{grid_id}', '{grid_element_id}');
    Tree2 = Tree2.DataFrame()

    #Merging elements that come one following the other
    BB = Tree2.merge(Tree2, left_on='terminal2_cn', right_on='terminal1_cn',suffixes=('_upper', '_lower'))

    #Tracing elements where the phases change from ABC to non-ABC
    bif = BB[(BB['phases_upper']=='ABC') & (BB['phases_lower']!='ABC')]
    bif = bif.reset_index()
    return bif


In [68]:
clusfind(grid_element_id='L7-97438')

,index,grid_element_id_upper,terminal1_cn_upper,terminal2_cn_upper,phases_upper,grid_element_id_lower,terminal1_cn_lower,terminal2_cn_lower,phases_lower
0,420,L7-100124,VCN.L7-100126,VCN.L7-100124,ABC,L7-137713,VCN.L7-100124,VCN.L7-137713,C
1,422,L7-100126,VCN.L7-37052,VCN.L7-100126,ABC,7831,VCN.L7-100126,VCN.7831,B
2,424,L7-100128,VCN.L7-100134,VCN.L7-100128,ABC,48331,VCN.L7-100128,VCN.48331,B
3,428,L7-100132,VCN.L7-100131,VCN.L7-100132,ABC,L7-100133,VCN.L7-100132,VCN.L7-100133,C
4,434,L7-100136,VCN.L7-101541,VCN.L7-100136,ABC,L7-138029,VCN.L7-100136,VCN.L7-138029,BC
5,687,L7-37052,VCN.L7-100134,VCN.L7-37052,ABC,120341,VCN.L7-37052,VCN.120341,A
6,690,L7-41560,VCN.L7-100124,VCN.L7-41560,ABC,L7-137915,VCN.L7-41560,VCN.L7-137915,A
7,691,L7-41560,VCN.L7-100124,VCN.L7-41560,ABC,L7-139409,VCN.L7-41560,VCN.L7-139409,C
8,694,L7-97438,VCN.L7-97435,VCN.L7-97438,ABC,26033,VCN.L7-97438,VCN.26033,B


In [209]:
def laminar(grid_id = 'North Central Zone', grid_element_id = 'L7-97438', level = '0', lami = []):
    
    #print(grid_element_id)
    lami_bran = []

    CC = branchfind(grid_id=grid_id, grid_element_id=grid_element_id)
    DD = clusfind(grid_id=grid_id, grid_element_id=grid_element_id)
    
    if len(DD)==0:
        return
    
    lami_clus = DD['grid_element_id_lower'].tolist()
    #print(lami_clus)
    lami_bran.append(lami_clus)
    lami_bran.append(level)
    #print(lami_bran)
    lami.append(lami_bran)
    #print(lami)
    
    ii = 0
    
    try:
        for jj in CC['grid_element_id']:
            level_new = level + str(ii)
            laminar(grid_element_id = jj, level = level_new, lami = lami)
            ii+=1
    except TypeError:
        pass
    
    return lami

In [210]:
lami2 = laminar()

In [211]:
lami2

[[['L7-137713',
   '7831',
   '48331',
   'L7-100133',
   'L7-138029',
   '120341',
   'L7-137915',
   'L7-139409',
   '26033'],
  '0'],
 [['48331', 'L7-100133'], '00'],
 [['L7-100133'], '001'],
 [['L7-137713', '7831', '120341', 'L7-137915', 'L7-139409'], '01']]